### Used directories:

#### Finance file locations:
'\\\\sigltd.co.uk\SIG_net\Departments\Protection\Pet\Loss Ratios\March 2024'

#### Postcode data (can be moved to Finance dir)
'\\\\coveainsurance.co.uk\data\Shared Data\BI Reporting\BI Function\Power BI\\2. Offline Data Sources'

#### Claims postcode lookup:
'\\\\sigltd.co.uk\SIG_net\Departments\Protection\Pet\Loss Ratios\MDA File prep'


In [ ]:
import os
import pandas as pd
import time
import datetime
import numpy as np
from dateutil.relativedelta import relativedelta #to add months to date
import re
# for converting excel date number to date
import xlrd 

# for testing function performance
from IPython.display import clear_output

# for claims postcode
import pyodbc


In [ ]:
directory = '\\\\sigltd.co.uk\SIG_net\Departments\Protection\Pet\Loss Ratios\March 2024'

for filename in os.listdir(directory):
    # Check if file is an Excel file and meets the name condition
    if filename.endswith('.xlsx') and not filename == 'Overall Loss ratio.xlsx':                  # and filename != 'Overall Loss ratio.xlsx'
        # Construct full file path
        file_path = os.path.join(directory, filename)
        #print(filename, time.ctime(os.path.getmtime(file_path)))
        print(filename, datetime.datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d %H:%M:%S'))



In [ ]:
dataframes = {}                                                                                 #Better to use a name such as dataframes_dict 

for filename in os.listdir(directory):
    # Check if file is an Excel file and meets the name condition
    if filename.endswith('.xlsx') and not filename == 'Overall Loss ratio.xlsx':                #Repeating above
        # Construct full file path
        file_path = os.path.join(directory, filename)
        print(filename)
        xls = pd.ExcelFile(file_path)
        # Iterate over all sheets in the Excel file
        for sheet_name in xls.sheet_names:
            sheet_name_lower = str.lower(sheet_name)
            # Check if sheet name meets the condition
            if sheet_name_lower.endswith('data') or \
                'fees' in sheet_name_lower or \
                (filename.startswith('IYP Written Data') and sheet_name == 'Sheet1'):
                # Load the data from the sheet into a dataframe
                df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
                # Check if the sheet is empty
                if df.empty:
                    continue
                # Check if the first cell is empty and use the appropriate row for headers
                headers = df.iloc[1] if pd.isnull(df.iloc[0, 0]) else df.iloc[0]
                df = pd.DataFrame(df.values[2:], columns=headers) if pd.isnull(df.iloc[0, 0]) else pd.DataFrame(df.values[1:], columns=headers)
                # Add file name and date modified to df
                df['filename'] = filename
                df['filemodified'] = datetime.datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d %H:%M:%S')
                # Store the dataframe in the dictionary
                dataframes[filename.replace('.xlsx', '') + '_' + sheet_name] = df
                print(filename, sheet_name, len(df.index))

In [ ]:
tableSchemaPath = '\\\\sigltd.co.uk\SIG_net\Departments\Protection\Pet\Loss Ratios\MDA File prep\PetLossRatio_TableSchema.csv'

tableSchema = pd.read_csv(tableSchemaPath, header = 0)

colsToDrop = tableSchema[tableSchema.Exclude == True]['ModelColumnName'].tolist()

# Create a dictionary for column name transformations
transform_dict = pd.Series(tableSchema.ModelColumnName.values, index=tableSchema.ColumnName).to_dict()

# Create a list of columns to exclude
exclude_list = tableSchema.loc[tableSchema['Exclude'] == True, 'ModelColumnName'].unique().tolist()

# Create a list of columns to convert to date
dateColumns = tableSchema[(tableSchema.DataType == 'date')]["ModelColumnName"].tolist()
dateColumns = list(set(dateColumns))

# Create list of Dimension columns to replace null with UNKNOWN
colsCleanForDim = tableSchema[
    (tableSchema.CleanForDim == True) & 
    (tableSchema.DataType != 'date' )]['ModelColumnName'].tolist()


def fnTransformHeaders(df):
    # Rename the columns using the transformation dictionary
    df.rename(columns=transform_dict, inplace=True)
    return df

def fnRemoveColumns(df):
    # Remove the columns in the exclude list
    df.drop(columns=exclude_list, errors='ignore', inplace=True)
    return df

def fnPrepDimCols(df):
    for col in df.columns:
        if col in colsCleanForDim:
            df.loc[:, col] = df.loc[:, col].apply(lambda x: 'UNKNOWN' if pd.isnull(x) or x == '' else x)
    return df

    
def convert_string_to_date(input_string):
    #check for excel number dates:
    if type(input_string) == int:
        return datetime.datetime(1899, 12, 30) + datetime.timedelta(days=int(input_string))
    #check different date formats:
    formats = [
        "%d/%m/%Y",
        "%d-%m-%Y",
        "%d-%m-%Y %H:%M:%S",
        "%Y-%m-%d",
        "%Y/%m/%d",
        "%Y-%m-%d %H:%M:%S",
        "%Y/%m/%d %H:%M:%S",
        "%d.%m.%Y",
        "%d/%m.%Y"
    ]
    for fmt in formats:
        try:
            date_obj = datetime.datetime.strptime(str(input_string), fmt)
            return date_obj
        except ValueError:
            pass
    
    #known issues
    errorDate =  '00/01/2020','02/012022','02/09/201807/2019','13/111/2019','14/06+/2021','14/12/20190','21/10/20189','25/17/2020','27/07/20221','27/11/20201','29/02/2019','29/02/2021','29/02/2022','30/042021','30/09/20232','31/02/2022','31/04/2019','31/04/2022','31/04/2023','31/06/2019','31/06/2020','31/06/2023','31/09/2021','31/09/2023','31/11/2020','31/11/2023','31/17/2021','65/03/2022','22/09/202','05/09/023','23/09/202'
    fixedDate = '2020-01-01','2022-01-02','2018-09-02','2019-11-13','2021-06-14','2019-12-14','2018-10-21','2020-10-25','2021-07-27','2020-11-27','2019-02-28','2021-02-28','2022-02-28','2021-04-30','2023-09-30','2022-02-28','2019-04-30','2022-04-30','2023-04-30','2019-06-30','2023-06-30','2023-06-30','2021-09-30','2023-09-30','2020-11-30','2023-11-30','2021-07-31','2022-03-16','2022-09-22','2023-09-05','2022-09-23'
    fixedDate = [datetime.datetime.strptime(d, '%Y-%m-%d') for d in fixedDate]
    
    if str(input_string) in errorDate:
        return fixedDate[errorDate.index(input_string)]
        
    return None

def fnConvertDates(df):
    for col in df.columns:
        if col in dateColumns:
            print('Converting dates:', key, col)
            df.loc[:, col] = df.loc[:, col].apply(convert_string_to_date)
    return df


def fnConvertDates(df):
    for col in df.columns:
        if col in dateColumns:
            print('Converting dates:', key, col)
            df.loc[:, col] = df.loc[:, col].apply(convert_string_to_date)
    return df

def fnIntersectCols(df, columns):
    df_cols = [val for val in df.columns if val in columns]
    return df_cols

### Setup save func to save in notebook directory
def write_df_to_csv(df, filename):
    df.to_csv(filename, encoding = 'utf-8-sig', index = False)


In [ ]:
# check if will work faster of fnConvertDates
#for key, df in dataframes.items():
#    df_dateColumns = [ val for val in dateColumns if val in df.columns]
#    df.loc[:, df_dateColumns] = df.loc[:, df_dateColumns].apply(convert_string_to_date
#    print(key, df_dateColumns)
    

In [ ]:
# Format headers, remove columns, format dates, remove rows where reporting_month is null
colsToInt = list(set(tableSchema[tableSchema.DataType == 'int']['ModelColumnName']))

for key, df in dataframes.items():
    # Transform headers
    df = fnTransformHeaders(df)

    # Remove columns
    df = fnRemoveColumns(df)

    # Convert/clean dates
    df = fnConvertDates(df)
    
    # Replace empty values with NaN
    df['reporting_month'].replace('', np.nan, inplace=True)
    
    # Drop rows with null values in the reporting_month column
    df.dropna(subset=['reporting_month'], inplace=True)

    # Format nulls on columns to be used as dimensions
    df = fnPrepDimCols(df)
    
    # Fix int on YOA and YOL

    dfCols = fnIntersectCols(df, colsToInt)
    df.loc[:,dfCols] = df.loc[:,dfCols].astype(int)



In [ ]:
### check for files with no affinity
for key, df in dataframes.items():
    if 'affinity' not in df.columns:
        print(key)
        #[affinityAll.add(x) for x in affinityDf]
        #df[key].loc[:, 'affinity'] = 'INSURE YOUR PAWS'


In [ ]:
### dataframe with missing affinity

dfma = 'IYP Written Data_Sheet1'
dataframes[dfma]['affinity'] = 'IYP'

In [ ]:
### get clean list of affinities for dim

affinityAll = set()
affinityDf = set()
for key, df in dataframes.items():
    if 'affinity' in df.columns:
        affinityDf = set(df.affinity.tolist())
        [affinityAll.add(x) for x in affinityDf]
        #df[key].loc[:, 'affinity'] = 'INSURE YOUR PAWS'
        
print(affinityAll)


affinityAllclean = [x.upper().replace('-','').replace('  ', ' ') for x in affinityAll]
affinityAllclean = [x.replace('VETSMEDICOVER', 'VMC') for x in affinityAllclean]
affinityAllclean = [x.replace('INSURE YOUR PAWS', 'IYP') for x in affinityAllclean]
affinityAllclean = [x.replace('LIFETIME PET COVER', 'LIFETIME') for x in affinityAllclean]
affinityAllclean = [x.replace('LPC', 'LIFETIME') for x in affinityAllclean]

affinityName = []
affinityTPL = []

for a in affinityAllclean:
    if 'TPL' in a:
        affinityName.append(a.replace('TPL','').strip())
        affinityTPL.append('TPL')
    else:
        affinityName.append(a.strip())
        affinityTPL.append('NonTPL')

affinities = pd.DataFrame({
    'affinity': list(affinityAll),
    #'affinity_clean': affinityAllclean,
    'affinity_name': affinityName,
    'affinity_tpl': affinityTPL
})


affinities


In [ ]:
### VMC Written and Paid data in single table, requires splitting

vmc_premium = 'VMC Loss Ratio Data_Premium Data'
vmc_df = ''
n = 0
for key, df in dataframes.items():
    if key.endswith(vmc_premium):
        vmc_df = key
        n += 1
if n == 1:
    VMCpremium = dataframes[vmc_df]
    dataframes[vmc_df + '_Written'] = VMCpremium[VMCpremium.transaction_group == 'Written']
    dataframes[vmc_df + '_Paid'] = VMCpremium[VMCpremium.transaction_group == 'Paid']
else:
    print('multiple vmc premium tables!')
    

### Written to 'Earned' based on Reporting Month

Where paid data is not available, written data is exploded into pseudo-earned.
Replicating logic from finance reporting.
Change to Earned GWP would require converting all premium, and abandoning use of paid.

Rules applied:

Split written dataframe into two sets:
*  <b>Keep as is</b>: YEAR(Reporting Month) > YOA -- Assuming cancellations, not to be earned out
*  <b>Expanded</b>: YEAR(Reporting Month) <= YOA -- Assuming NB/RNL, to earn out

#### Expanded
For each reporting month create list of current and Next 11 months.
Join on reporting month
Multiply all columns listed as currency type in TableSchema by 1/12

#### Combine tables 'Keep as is' and 'Expanded'

### Data requiring converting:

#### THISTLE
Base logic using Written data

#### VMC
All written premium where reporting month is on or before 2022-07-01 and collector is 'Premium Credit'
Original VMC BDX file combines both paid and written, it is split out, but for consistency with THISTLE I'm using the \__Premium Data_ file

#### LIFETIME
At the moment all written data has to be converted, with no cap on YEAR(reporting month) <= YOA.
Likely to be reviewed in July





In [ ]:
def fnDivideCurrForEarned(df, n):
    #get all currency cols
    try:
        currCols = tableSchema[ (tableSchema.DataType == 'currency') & ( tableSchema.Exclude == False )]['ModelColumnName'].tolist()
    except ValueError:
        'is tableSchema loaded?'
        return
    #get currency cols in df
    df_currCols = [val for val in df.columns if val in currCols]
    df.loc[:, df_currCols] = df.loc[:, df_currCols].div(n)
    return

def fnExpandOnReportingMonth(df):
    
    if 'reporting_month' not in df.columns:
        print('no reporting_month column')
        return
    
    expandby = pd.DataFrame(set(df.reporting_month), columns = ["reporting_month"])
    expandby['key'] = 0

    expand_df = pd.DataFrame([x for x in range(12)], columns = ['months'])
    expand_df['key'] = 0

    expandby = expandby.merge(expand_df, on = 'key', how = 'outer')
    expandby['earned_month'] = expandby.apply(
        lambda row: row.reporting_month + relativedelta(months = row.months),
        axis = 1)
                                    
    expandby = expandby.drop(['key','months'], axis = 1)

    df = df.merge(expandby, on = 'reporting_month', how = 'inner')
    df = df.drop(['reporting_month'], axis = 1)
    df = df.rename(columns={'earned_month': 'reporting_month'})
    
    return df

def fnGetDataForEarned(endswith, contains):
    n_df = [x for x in dataframes.keys() if (x.endswith(endswith)) and (contains in x.upper())]
    if len(n_df) == 1:
        df_w = dataframes[n_df[0]]
    else:
        print(f'this many {len(n_df)} vmc dataframes matching') 
        return None
    return df_w
    
#learn how to use classes
def fnGetKeyForEarned(endswith, contains):
    n_df = [x for x in dataframes.keys() if (x.endswith(endswith)) and (contains in x.upper())]
    if len(n_df) == 1:
        return n_df[0]
    else:
        print(f'this many {len(n_df)} vmc dataframes matching')                                           
        return None
    
                                             
                                             
                                        

In [ ]:
### Get written files to update

# VMC
vmc_w = fnGetDataForEarned('_Premium Data_Written', 'VMC')
vmc_w = vmc_w[vmc_w.collector == 'Premium Credit']

# THISTLE
th_w = fnGetDataForEarned('_Written Premium Data', 'THISTLE')

# LIFETIME
lt_w = fnGetDataForEarned('_Premium Data', 'LIFETIME')

print(f'VMC rows: ', len(vmc_w))
print(f'THISTLE rows: ', len(vmc_w))
print(f'LIFETIME rows: ', len(lt_w))



In [ ]:
# Expand VMC
vmc_cutoff = datetime.datetime(2022,7,1)

#vmc_cutoff + relativedelta(months=1)
#set(vmc_w.yoa)
#[x.year for x in list(set(vmc_w.reporting_month))]

vmc_expand = vmc_w[
    ( vmc_w.yoa == [ x.year for x in vmc_w.reporting_month] ) & 
    ( vmc_w.reporting_month <= vmc_cutoff ) 
]

vmc_keep = vmc_w[
    ( vmc_w.yoa != [ x.year for x in vmc_w.reporting_month] ) |
    ( vmc_w.reporting_month > vmc_cutoff ) 
]

vmc_expand = fnExpandOnReportingMonth(vmc_expand)

fnDivideCurrForEarned(vmc_expand, 12)

vmc_earned = pd.concat([vmc_keep, vmc_expand])

vmc_earned.transaction_group = 'Earned'

# Check
print(sum(vmc_earned.allianz_net))
print(sum(vmc_w.allianz_net))

In [ ]:
# LIFETIME
### temporarily explode all. to be confirmed

lt_expand = lt_w

lt_expand = fnExpandOnReportingMonth(lt_expand)

fnDivideCurrForEarned(lt_expand, 12)

lt_earned = lt_expand

lt_earned.transaction_group = 'Earned'

# Check
print(sum(lt_earned.allianz_net))
print(sum(lt_w.allianz_net))





In [ ]:
# THISTLE

thistle_expand = th_w[
    ( th_w.yoa == [ x.year for x in th_w.reporting_month] )
]

thistle_keep = th_w[
    ( th_w.yoa != [ x.year for x in th_w.reporting_month] )
]

thistle_expand = fnExpandOnReportingMonth(thistle_expand)

fnDivideCurrForEarned(thistle_expand, 12)

thistle_earned = pd.concat([thistle_keep, thistle_expand])

thistle_earned.transaction_group = 'Earned'

# Check
print(sum(th_w.allianz_net), len(th_w))
print(sum(thistle_earned.allianz_net), len(thistle_earned))

In [ ]:
### Add VMC Earned to df dict
vmc_earned_name = fnGetKeyForEarned('_Premium Data_Written', 'VMC').replace('_Premium Data_Written', '_Earned Premium Data')
dataframes[vmc_earned_name] = vmc_earned

### Add LIFETIME Earned to df dict
lifetime_earned_name = fnGetKeyForEarned('_Premium Data', 'LIFETIME').replace('_Premium Data', '_Earned Premium Data')
dataframes[lifetime_earned_name] = lt_earned

### Add THISTLE Earned to df dict
thistle_earned_name = fnGetKeyForEarned('_Written Premium Data', 'THISTLE').replace('_Written Premium Data', '_Earned Premium Data')
dataframes[thistle_earned_name] = thistle_earned


In [ ]:
### Create mapping table to combine written, paid, and claims tables

concat_check = ['written','claims','paid','fees','Lifetime Pet Cover Loss Ratio Data_Premium Data', 'earned']
concat_file = ['written','claims','paid','fees','written','paid']
concat_df = pd.DataFrame(columns=['key','file'])
for key, df in dataframes.items():
    for n in range(len(concat_check)):
        if str.lower(concat_check[n]) in str.lower(key):
            concat_df = concat_df.append({'key': key, 'file': concat_file[n]}, ignore_index = True)
    if not (key in concat_df.key.values):
        concat_df = concat_df.append({'key': key, 'file': 'other'}, ignore_index = True)
        

concat_df.sort_values(by = ['file'])

In [ ]:
### Combine all written, paid, and claims tables

dataframes_out = {}
dfs = list(set(concat_file))
for c in dfs:
    dfs_list = concat_df[concat_df.file == c]['key'].tolist()
    dfs_concat = pd.concat([dataframes[i] for i in dfs_list])
    dataframes_out[c] = dfs_concat

    

In [ ]:
# Add affinity_name (all) and affinity_tpl (claims only)

for key, df in dataframes_out.items():
    if key == 'claims':
        dataframes_out[key] = df.merge(affinities, on = 'affinity', how = 'inner')
    else:
        dataframes_out[key]  = df.merge(affinities[['affinity','affinity_name']], on = 'affinity', how = 'inner')
    


In [ ]:
# check affinity
test1 = pd.DataFrame()
for key, df in dataframes_out.items():
    if key == 'claims':
        dfa = df[['affinity','affinity_name','affinity_tpl']].copy()
    else:
        dfa = df[['affinity','affinity_name']].copy()
    dfa.drop_duplicates(inplace = True)
    dfa.loc[:,'key'] = key
    test1 = pd.concat([test1, dfa], axis = 0, ignore_index = True)
    
#test2 = test1[['affinity_name']].copy()

pivot_test = test1.pivot_table(index = 'affinity_name', columns='key', values='affinity', aggfunc='count')
pivot_test['TPL'] = test1[test1.affinity_tpl == 'TPL'].groupby('affinity_name')['affinity_tpl'].count()
pivot_test

In [ ]:
# Add policyNkey. Based on policy_no + affinity_name + yoa
def fnLambdaPolicyNkey(policy_no, affinity_name, yoa):
    if pd.isna(policy_no) or pd.isna(affinity_name) or pd.isna(yoa):
        return 'UNDEFINED'
    elif str(policy_no).lower().strip() == 'bdx':
        return 'UNDEFINED'
    else:
        return '+'.join([str(policy_no).strip(),str(affinity_name),str(yoa)])


def fnAddPolicyNkey(df):
    policyNkeyCols = ['policy_no','affinity_name','yoa']
    key_cols = fnIntersectCols(df, policyNkeyCols)
    if len(key_cols) != len(policyNkeyCols):
        df.loc[:, 'policyNkey'] = 'UNDEFINED'
    else:
        df.loc[:, 'policyNkey'] = df.apply(lambda x: fnLambdaPolicyNkey(x.policy_no, x.affinity_name, x.yoa), axis = 1)
    return None


In [ ]:
for key, df in dataframes_out.items():
    fnAddPolicyNkey(df)
    print(key, df.policyNkey[:3].tolist())


In [ ]:
# Add developmentID

#def fnAddDevelopmentID(df):
#    df.loc[:, 'developmentid'] = df.apply(lambda x: )
 
def fnLambdaDevelopmentID(yoa, reporting_month):
    if yoa is None or yoa > 2100:
        return -1
    else:
        return yoa * 1000 + ( reporting_month.year - yoa ) * 12 + reporting_month.month
    
def fnAddDevelopmentID(df):
    df.loc[:, 'developmentid'] = df.apply(lambda x: fnLambdaDevelopmentID(x.yoa, x.reporting_month), axis = 1)

#dataframes_out['written'].apply(lambda x: fnLambdaDevelopmentID(x.yoa, x.reporting_month), axis = 1)

for key, df in dataframes_out.items():
    fnAddDevelopmentID(df)
    print(key)

In [ ]:
# Add pet age

def calculate_age(born, asat):
    return asat.year - born.year - ((asat.month, asat.day) < (born.month, born.day))


# If pet_dob is null: return -1
# If start_date is available: calculate the age in years based on the difference
# If not, use YOA start date
# If age is negative or null: return -1

def fnReturnPetAge(pet_dob, start_date, yoa):
    age = -1
    if pd.isnull(pet_dob):
        return age
    elif not pd.isnull(start_date):
        age = calculate_age(pet_dob, start_date)
    else:
        age = calculate_age(pet_dob, datetime.date(yoa, 1, 1))

        
    if age < 0 or pd.isnull(age):
        return -1   
    else:
        return age
    
    
ageCalcList = ['pet_dob','start_date','yoa']

for key, df in dataframes_out.items():
    if 'pet_dob' in df.columns:
        print(key, fnIntersectCols(df, ageCalcList))
        df.loc[:, 'pet_age'] = df.apply(lambda x: fnReturnPetAge(x.pet_dob, x.start_date, x.yoa), axis = 1 )

        

In [ ]:
# Format Pet Type
# Format Breed: add dictionary 
# Add pet key (petNkey = pet_type + breed)
# Clean sales channel

def fnFormatDimText(value):
    if pd.isna(value) or value == 0:
        value = 'Unknown'
    value = value.title().strip()
    return value

breedDict = {
    'Labrador': 'Labrador Retriever',
    'Labrador (Retriever)': 'Labrador Retriever',
    'Golden Labrador': 'Golden Labrador Retriever',
    'Chocolate Labrador' : 'Labrador Retriever',
    'Retriever (Labrador)': 'Labrador Retriever',
    'Retriever (Labrador)': 'Labrador Retriever',
    'Labrador (Retriever) Cross': 'Labrador Retriever Cross',
    'Non Pedigree Rabbit': 'Non-Pedigree',
    'Non-Pedigree (Cat)': 'Non-Pedigree',
    'Non-Pedigree (Dog)': 'Non-Pedigree'    
}


def fnFormatPetBreed(breed):
    breed = fnFormatDimText(breed)
    if breed in breedDict:
        breed = breedDict[breed]
    return breed

for key, df in dataframes_out.items():
    petNkeylist = ['pet_type','breed']
    
    if 'pet_type' in df.columns:
        df.pet_type = [ fnFormatDimText(x) for x in df.pet_type ]
    if 'breed' in df.columns:
        df.breed = [ fnFormatPetBreed(x) for x in df.breed ]
    
    if all(e in df.columns for e in petNkeylist):
        df['petNkey'] = df.apply(lambda x: '+'.join([x.pet_type, x.breed]), axis = 1)
    
    if 'sales_channel' in df.columns:
        df.loc[:, 'sales_channel'].mask(df.loc[:, 'sales_channel'] == 0, inplace = True)
        df.loc[:, 'sales_channel'].fillna('Unknown', inplace = True)
        df.loc[:, 'sales_channel'] = df.sales_channel.apply(lambda x: x.title())
        
        

        



In [ ]:
# Add sales_channel to claims based on written and policy data

scl1 = dataframes_out['written'][['policy_no','sales_channel']].copy()
scl2 = dataframes_out['paid'][['policy_no','sales_channel']].copy()

scl = pd.concat([scl1, scl2])

del scl1, scl2

scl.drop_duplicates(subset = ['policy_no'], inplace = True, keep = 'last')

scl.dropna(subset = ['policy_no'])

dfc = dataframes_out['claims']

dfcSC =pd.merge(dfc, scl, how = 'left', on = 'policy_no')
dfcSC.loc[:,'sales_channel'].fillna('Unknown', inplace = True)

dataframes_out['claims'] = dfcSC


### Other dimensions

In [ ]:
steps = 17
step = 50

purchase_price_dict = {}

purchase_price_buckets = [[-1,'UNKNOWN']]

for n in range(steps):
    lb = n * step
    up = (n + 1) * step - 1
    purchase_price_buckets.append([lb, '-'.join([str(lb), str(up)])])

purchase_price_buckets.append([purchase_price_buckets[-1][0], str(purchase_price_buckets[-1][0]) + '+'])

for x in purchase_price_buckets:
    purchase_price_dict[x[0]] = x[1]


def fnConvertPurchasePrice(price):
    try:
        price_int = int(float(price))
    except ValueError:
        price_int = None

    if price_int is not None:
        return steps * step if price_int > steps * step else price_int - price_int % step
    else:
        try:
            if "-" in str(price):
                try:
                    return int(re.sub(r'[^0-9]', '',price.split("-")[0]))
                except ValueError:
                    return -1
            else:
                return -1
        except ValueError:
            print(price)

def fnLookupPurchaseRange(price):
    if price in purchase_price_dict:
        return purchase_price_dict[price]
    else:
        'UNKNOWN'
    

purchase_price_dict

In [ ]:
for key in ['written','paid']:
    x = dataframes_out[key]
    convertedPrices = 
    x.loc[:,'purchase_price_bucket_id'] = x.apply(lambda y: fnConvertPurchasePrice(y.purchase_price), axis = 1)
    x.loc[:,'purchase_price_bucket'] = x.apply(lambda y: fnLookupPurchaseRange(y.purchase_price_bucket_id), axis = 1)
    
 

In [ ]:
# Try match purchase_price to claims data
# Tried policyNkey + pet_name. Resulted in 5/12 match
# 800 policyNkeys on Claims side have two pet name instances.
# If we discard this detail, and use policyNkey, match rate increases to 10/12

policy_price_key = pd.DataFrame()
for key in ['written','paid']:
    y = dataframes_out[key][['policy_no', 'policyNkey', 'purchase_price_bucket_id']].copy()
    y = y.dropna(subset = ['policy_no'])
    #y['policyNkeyPetName'] = y.apply(lambda z: '+'.join([z.policyNkey, str(z.pet_name)]), axis = 1)
    y = y[['policyNkey', 'purchase_price_bucket_id']]
    #y.drop_duplicates(subset = ['policyNkeyPetName'], keep = 'first', inplace = True)
    y.drop_duplicates(subset = ['policyNkey'], keep = 'first', inplace = True)
    policy_price_key = pd.concat([policy_price_key, y])

#x.drop_duplicates(subset = ['policyNkeyPetName'], keep = 'first', inplace = True)
policy_price_key.drop_duplicates(subset = ['policyNkey'], keep = 'first', inplace = True)
del y
          
policy_price_key



In [ ]:
#dataframes_out['claims']['policyNkeyPetName'] = dataframes_out['claims'].apply(
#    lambda z: '+'.join([z.policyNkey, str(z.pet_name)]), axis = 1)
if 'purchase_price_bucket_id' not in dataframes_out['claims'].columns:
    dataframes_out['claims'] = pd.merge(dataframes_out['claims'], policy_price_key, on = ['policyNkey'], how = 'left')
    dataframes_out['claims'].loc[:,'purchase_price_bucket_id'].fillna(-1, inplace = True)
    dataframes_out['claims'].loc[:,'purchase_price_bucket'] = dataframes_out['claims']['purchase_price_bucket_id'].apply(lambda y: fnLookupPurchaseRange(y))

#list(set(dataframes_out['claims']['purchase_price_bucket_id']))



### Cover Type cleaning (in progress)

In [ ]:
cover_type = pd.DataFrame()
cover_type_add = pd.DataFrame()
for key, df in dataframes_out.items():
    if 'cover_type' in df.columns:
        cover_type_add = df[['affinity_name', 'cover_type']]
        cover_type_add = cover_type_add[[(not str(x).startswith('PET')) or str(x).startswith('PETtrac') for x in cover_type_add.cover_type]]
        cover_type = cover_type.append(cover_type_add, ignore_index = True)
        cover_type.drop_duplicates(inplace = True)

#cover_type -= cover_type_remove
#cover_type -= set([0, np.NaN, 'UNKNOWN', 'Missing'])

cover_type.groupby(by=['affinity_name'], as_index = False)['cover_type'].count()

In [ ]:
cover_type_dict = {}
cover_type_dict = {
    'IYP': {
        'accident': 'Accident',
        'maximum': 'Maximum',
        'lifetime': 'Lifetime'
    },
    'VMC': {
        'diamond': 'Diamond',
        'gold':'Gold',
        'platinum': 'Platinum',
        'silver': 'Silver',
        '10k': 'Diamond',
        '6k': 'Platinum',
        '4k': 'Gold',
        '2k': 'Silver'
    },
    'THISTLE': {
        '1k': '£1,000',
        '3k': '£3,000',
        '6k': '£6,000',
        '7.5k': '£7,500',
        '12k': '£12,000',
        '£1,000': '£1,000',
        '£3,000': '£3,000',
        '£6,000': '£6,000',
        '£7,500': '£7,500',
        '£12,000': '£12,000'
    }
}

def fnCoverType(affinity, cover_type):
    cover_type_key = str(cover_type).lower().strip()
    cover_type_desc = 'UNKNOWN'
    if affinity in ['IYP','VMC']:
        for cover_type_item in cover_type_dict[affinity]:
            if cover_type_key.startswith(cover_type_item):
                cover_type_desc = cover_type_dict[affinity][cover_type_item]
    elif affinity in ['THISTLE']:
        for cover_type_item in cover_type_dict[affinity]:
            if  cover_type_item in cover_type_key:
                cover_type_desc = cover_type_dict[affinity][cover_type_item]
    elif affinity == 'RSPCA':
        cover_type_desc = cover_type_key.title()
    elif affinity == 'LIFETIME':
        cover_type_desc = cover_type_key.replace('(legacy)', '').strip().title()

    return cover_type_desc

cover_type['cover_type_desc'] = cover_type.apply(lambda x: fnCoverType(x.affinity_name, x.cover_type), axis = 1)
#cover_type.head()


cover_type_lookup = {}
cover_type_lookup_item = {}

for affinity in set(cover_type.affinity_name):
    for ct in set(cover_type[cover_type.affinity_name == affinity]['cover_type'].tolist()):
        cover_type_lookup[affinity][cover_type_lookup_item[ct]] = fnCoverType(affinity, ct)


In [ ]:
'10K'.startswith('10K')

### Postcode

#### Claims - Add from CSV lookup
Customer postcode is not included in the claims data\
Could be joined from policy data or retrieved from Aquarius \
Aquarius seems better

#### Written/Paid - Format


In [ ]:
# Get claim_id, postcode data from CSV (SQL output from AQ as postcode is not available in AllClaims data)
directory = '\\\\sigltd.co.uk\SIG_net\Departments\Protection\Pet\Loss Ratios\MDA File prep'
filename = 'AllClaimsPostcode.csv'

allClaimsPostcode = pd.read_csv(os.path.join(directory, filename), header = None)
allClaimsPostcode.columns = ['claim_id','post_code']

allClaimsPostcode.head()

In [ ]:
regex =re.compile(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})')

postcodeErrors = {
    'DT4OPL':'DT4 0PL',
    'WD5OGE': 'WD5 0GE'
}
def postcodeRegEx(postcode, errorValue = 'INVALID'):
    if pd.isna(postcode):
        return 'UNKNOWN'
    postcode = str(postcode).replace('-', '').replace(' ','').upper()
    if postcode == 'IRELAND':
        'IRELAND'
    matchobj = regex.search(postcode)
    if matchobj:
        matchobj = matchobj.group(0).replace(" ", "")
        return matchobj[:-3] + ' ' + matchobj[-3:]
    elif postcode in postcodeErrors:
        return postcodeErrors[postcode]
    return errorValue


In [ ]:
# Merge AllClaimsPostcode into Claims dataframe
if 'post_code' not in dataframes_out['claims'].columns:
    claims = dataframes_out['claims']
    claims = pd.merge(claims, allClaimsPostcode, on = 'claim_id', how = 'left')
    dataframes_out['claims'] = claims

dataframes_out['claims'][['claim_id','post_code']].head()

In [ ]:
# Merge AllClaimsPostcode into Claims dataframe


# Apply formatting

for key, df in dataframes_out.items():
    if 'post_code' in df.columns:
        df.loc[:, 'post_code_orig'] = df.post_code
        df.loc[:, 'post_code'] = df.post_code.apply(lambda x: postcodeRegEx(x))
        df.loc[:, 'post_code_orig'] = df.apply(lambda x: x.post_code_orig if x.post_code == 'INVALID' else x.post_code, axis = 1)
        
        

In [ ]:
### Save each table
for key, df in dataframes_out.items():
    print(key + '.csv')
    write_df_to_csv(df, key + '.csv')
    

### Postcodes
Experian UK 7 data. Creates dimension table

In [ ]:
directory = '\\\\coveainsurance.co.uk\data\Shared Data\BI Reporting\BI Function\Power BI\\2. Offline Data Sources'

pc_extract = pd.read_csv(os.path.join(directory, 'PC_Extract.csv'))
pc_type = pd.read_csv(os.path.join(directory, 'PC_Types.csv'))
pc_group = pd.read_csv(os.path.join(directory, 'PC_Groups.csv'))

In [ ]:
pc_extract.columns = ['post_code_equivalent','post_code_flag','post_code','group_id','type_id']
pc_extract.loc[:,'group_key'] = pc_extract.apply(lambda x: '+'.join([x.group_id, str(x.type_id) ]), axis = 1)
pc_extract.head()
len(pc_extract)

In [ ]:
def fnSplitPostcode(postcode):
    postcode = postcode.replace(' ','')
    return postcode[:-3] + ' ' + postcode[-3:]

pc_extract.loc[:,'post_code'] = pc_extract.post_code.apply(lambda x: fnSplitPostcode(x))
pc_extract['district'] = pc_extract.post_code.apply(lambda x: x.split(' ')[0])

regex = re.compile(r'[^A-Z]')
pc_extract['post_code_area'] = pc_extract.district.apply(lambda x: regex.sub('',x[:2]))

In [ ]:
# Add missing postcodes from Claims, Written and Paid
pc_missing = dataframes_out['claims'].post_code.tolist()
pc_missing += dataframes_out['paid'].post_code.tolist()
pc_missing += dataframes_out['written'].post_code.tolist()
pc_missing = set(pc_missing)
pc_missing -= set(pc_extract.post_code)
pc_missing -= set(['INVALID','UNKNOWN'])
print(len(pc_missing))
cols = ['post_code_equivalent','post_code_flag','post_code','group_id','type_id','group_key','district','post_code_area']

df_missing = []
for pc in pc_missing:
    d = pc.split(' ')[0]
    a = regex.sub('', d[:2])
    l = [pc,None,pc,'U','99','U+99', d, a]
    df_missing.append(l)
    
df_missing = pd.DataFrame(df_missing, columns = cols)
pc_extract = pc_extract.append(df_missing, ignore_index = True)
print(len(pc_extract))


In [ ]:
pc_groups = pc_extract[['group_id','type_id']].drop_duplicates()
if 'group_desc' not in pc_groups.columns:
    pc_groups = pd.merge(pc_groups, pc_group, on = 'group_id', how = 'left')
    pc_groups = pd.merge(pc_groups, pc_type, on = 'type_id', how = 'left')
pc_groups.loc[:,'group_key'] = pc_groups.apply(lambda x: '+'.join([x.group_id, str(x.type_id)]), axis = 1)
pc_groups.drop(columns = ['group_id','type_id'], inplace = True)
pc_groups = pc_groups.drop_duplicates(subset = ['group_key'])
pc_groups.head()

In [ ]:
pc_areas_xls = pd.ExcelFile(os.path.join(directory, 'PostcodeDistricts.xlsx'))
pc_areas_xls.sheet_names

dfDistrict = pd.read_excel(pc_areas_xls, sheet_name='Districts', header=0)
dfDistrict.drop(columns = ['DistrictCount','Area'], inplace = True)
dfDistrict.columns = ['district','county']

dfArea = pd.read_excel(pc_areas_xls, sheet_name = 'Area', header=0)
dfArea.columns = ['post_code_area','post_code_area_name','post_code_region']

dfArea.head()
#dfDistrict.head()

In [ ]:
print(len(pc_extract))
if 'county' not in pc_extract.columns:
    pc_extract = pd.merge(pc_extract, dfDistrict, on = 'district', how = 'left')
    
if 'post_code_area_name' not in pc_extract.columns:
    pc_extract = pd.merge(pc_extract, dfArea, on = 'post_code_area', how = 'left')
print(len(pc_extract))    

pc_extract.drop(columns = ['post_code_equivalent','post_code_flag'], inplace = True)

pc_extract = pd.merge(pc_extract, pc_groups, on = 'group_key', how = 'left')
pc_extract.rename(columns = {'group_desc': 'UK7_group_desc', 'type_desc': 'UK7_type_desc'}, inplace = True)

pc_extract.head()

In [ ]:
pc_u = pd.DataFrame([
    ['UNKNOWN', 'U', 99, 'U+99', 'UNKNOWN', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'],
    ['INVALID', 'U', 99, 'U+99', 'INVALID', 'Invalid', 'Invalid', 'Invalid', 'Invalid', 'Invalid', 'Invalid']
], columns = pc_extract.columns)


pc_extract = pc_extract.append(pc_u, ignore_index = True)
pc_extract[-2:]

In [ ]:
pc_extract = pc_extract.fillna('Unknown')
write_df_to_csv(pc_extract, 'postcodes.csv')